<a href="https://colab.research.google.com/github/dougscohen/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module1-rnn-and-lstm/LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
""" IMPORTS """

import requests
import re
import numpy as np
import random
import sys

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import LambdaCallback

In [0]:
url = 'https://www.gutenberg.org/files/100/100-0.txt'

response = requests.get(url)
text = response.text

In [0]:
len(text)

5777367

In [0]:
cleaned_text = text.replace('\r\n', ' ').replace('ï»¿', ' ').strip()

In [0]:
new_text = re.sub(r'[^a-zA-Z ]', '', cleaned_text)[:1000000]
new_text

'Project Gutenbergs The Complete Works of William Shakespeare by William Shakespeare  This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever  You may copy it give it away or reuse it under the terms of the Project Gutenberg License included with this eBook or online at wwwgutenbergorg  If you are not located in the United States youll have to check the laws of the country where you are located before using this ebook   Title The Complete Works of William Shakespeare  Author William Shakespeare  Release Date January  EBook  Last Updated November    Language English  Character set encoding UTF   START OF THIS PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE      The Complete Works of William Shakespeare    by William Shakespeare           Contents                   THE SONNETS                 ALLS WELL THAT ENDS WELL                 THE TRAGEDY OF ANTONY AND CLEOPATRA      

In [0]:
# Unique Characters
chars = list(set(new_text))

# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [0]:
len(chars)

53

In [0]:
# Create the sequence data

maxlen = 80
step = 1

encoded = [char_int[c] for c in new_text]

sequences = [] # Each element is 80 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  999920


In [0]:
len(sequences[0])

80

In [0]:
sequences[0]

[25,
 18,
 12,
 29,
 13,
 1,
 50,
 46,
 51,
 45,
 50,
 13,
 24,
 20,
 13,
 18,
 42,
 7,
 46,
 40,
 31,
 13,
 46,
 28,
 12,
 23,
 6,
 26,
 13,
 50,
 13,
 46,
 16,
 12,
 18,
 22,
 7,
 46,
 12,
 21,
 46,
 16,
 34,
 26,
 26,
 34,
 27,
 23,
 46,
 38,
 31,
 27,
 22,
 13,
 7,
 6,
 13,
 27,
 18,
 13,
 46,
 20,
 2,
 46,
 16,
 34,
 26,
 26,
 34,
 27,
 23,
 46,
 38,
 31,
 27,
 22,
 13,
 7,
 6,
 13]

In [0]:
# Create x & y

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [0]:
x.shape

(999920, 80, 53)

In [0]:
y.shape

(999920, 53)

In [0]:
# build the model: a single LSTM

model = Sequential([
    LSTM(128, input_shape=(maxlen, len(chars)), activation='tanh', recurrent_activation='sigmoid', dropout=0.2, recurrent_dropout=0, unroll=False, use_bias=True),
    Dense(len(chars), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(new_text) - maxlen - 1)
    
    generated = ''
    
    sentence = new_text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
# fit the model

shake_n_bake = model.fit(x, y,
                         batch_size=256,
                         epochs=20,
                         callbacks=[print_callback])

Epoch 1/20
3906/3906 [==============================] - ETA: 0s - loss: 2.3160
----- Generating text after Epoch: 0
----- Generating with seed: "our stomachs Will make whats homely savoury weariness Can snore upon the flint w"
our stomachs Will make whats homely savoury weariness Can snore upon the flint widlion thins     Thit in or cankerese th me thy shes thexgs     Coverint not thescance the suredihe ans comyacripling noug gr to lods     not you seie the boucbse Soctdee by toefhertans  he You eelio lo chene gyou shevilis     Oher hedgs PASooneure pIokeow   NURIIMSS Lould unowe sgom as cucaester Aspoult ang up Ind     Thisp comYous The porate Versiznte Sortahe not deefittones here thet wids ANTVO
3906/3906 [==============================] - 48s 12ms/step - loss: 2.3160
Epoch 2/20
3905/3906 [============================>.] - ETA: 0s - loss: 2.0398
----- Generating text after Epoch: 1
----- Generating with seed: "ride says the peacock Mistress that you know   Exeunt Antipholus of Syrac

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN